# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233841879444                   -0.50    7.0         
  2   -7.249853439303       -1.80       -1.40    1.0   7.70ms
  3   -7.251097537960       -2.91       -1.90    2.0   8.81ms
  4   -7.250994666915   +   -3.99       -1.90    2.0   9.38ms
  5   -7.251323551963       -3.48       -2.57    1.0   7.49ms
  6   -7.251338257283       -4.83       -3.26    1.0   7.69ms
  7   -7.251338707369       -6.35       -3.59    3.0   10.6ms
  8   -7.251338793640       -7.06       -4.11    2.0   9.47ms
  9   -7.251338797458       -8.42       -4.61    1.0   7.66ms
 10   -7.251338798647       -8.92       -5.13    3.0   10.4ms
 11   -7.251338798691      -10.36       -5.43    3.0   10.7ms
 12   -7.251338798702      -10.97       -5.88    1.0   8.14ms
 13   -7.251338798704      -11.71       -6.37    2.0   9.04ms
 14   -7.251338798705      -12.59       -6.74    2.0   9.85ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.043372318567918636
[ Info: Arnoldi iteration step 2: normres = 0.3791347963629421
[ Info: Arnoldi iteration step 3: normres = 0.7306850599951538
[ Info: Arnoldi iteration step 4: normres = 0.5287276007079457
[ Info: Arnoldi iteration step 5: normres = 0.5733070693884327
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.52e-01, 8.50e-02, 5.02e-01, 2.14e-01, 1.43e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3026888900699499
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.83e-02, 1.45e-01, 2.05e-01, 1.23e-01, 1.04e-01)
[ Info: Arnoldi iteration step 7: normres = 0.06662911806187036
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.23e-03, 9.88e-03, 9.98e-03, 3.50e-02, 4.53e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10829632098172595
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.82e-